In [1]:
import os
import json
import pandas as pd
from bs4 import BeautifulSoup
import re

DATA_DIR = "data/4chan"

In [2]:
list_wanted_boardnames = ['his']

text_list = []
for boardname in list_wanted_boardnames:
    thread_dp = os.path.join(DATA_DIR, f'boards/{boardname}/threads')
    for file in os.listdir(thread_dp):
        with open(os.path.join(thread_dp, file), "r") as f:
            dict_list = json.load(f)
            for dict_thread in dict_list:
                if 'sub' in dict_thread:
                    text_list.append(dict_thread['sub'])
                if 'com' in dict_thread:
                    text_list.append(dict_thread['com'])
text = "\n".join(text_list)

In [3]:
text = re.sub("&gt;&gt;\d{8}", "", text)
text = re.sub("<br><span class=\"quote\">&gt;", "", text)
text = re.sub("</span><br>", " ", text)



In [4]:
soup = BeautifulSoup(text,'html.parser')


In [5]:
from flair.data import Sentence
from flair.models.sequence_tagger_model import SequenceTagger



In [6]:
tagger = SequenceTagger.load("ner-fast")

sentence = Sentence(soup.get_text())
tagger.predict(sentence)




/home/luis/Proyectos/4chan-scraper/venv/lib/python3.8/site-packages/huggingface_hub/file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-11-08 09:49:12,241 loading file /home/luis/.flair/models/ner-english-fast/4c58e7191ff952c030b82db25b3694b58800b0e722ff15427f527e1631ed6142.e13c7c4664ffe2bbfa8f1f5375bd0dced866b8c1dd7ff89a6d705518abf0a611
2022-11-08 09:49:13,315 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [7]:

entities = {}
for span in sentence.get_spans("ner"):

    if span.tag not in entities:
        entities[span.tag] = {}
    if span.text in entities[span.tag]:
        entities[span.tag][span.text] += 1
    else:
        entities[span.tag][span.text] = 1

for label, value in entities.items():
    print(label)
    print(dict(sorted(value.items(), key=lambda item: item[1], reverse=True)))

LOC


TypeError: 'dict' object is not callable